# GEOG0027 (2022/2023) Spatial Filtering with Google Earth Engine (GEE)

This notebook uses the image exploration scripts from the [Intro Google Earth Engine](docs/Intro_to_GEE.ipynb) practical, and introduces spatial filtering functions to the images. First complete the introduction practical before attempting this one.

Again we recommend using UCL's [Jupyter Hub](https://jupyter.data-science.rc.ucl.ac.uk/) (VPN required) to work with this chapter. To download this notebook, to execute and modify the code, run the following command in a `Terminal`:

`wget https://github.com/UCL-EO/GEOG0027/raw/main/docs/SpatialFilteringGEE.ipynb`

As with the introduction practical, modify the local lon/lat coordinates to get the location you desire.

In [4]:
import geemap, ee, os, numpy
import ipyleaflet

loc_coords = [51.49163903397572, -0.08313179193795397] ### EDIT THIS LINE WITH YOUR COORDS, [Lat,Lon] format

Map = geemap.Map(center=loc_coords, zoom=9)
Map

Map(center=[51.49163903397572, -0.08313179193795397], controls=(WidgetControl(options=['position', 'transparen…

A basic Google Map-like interface should be displayed here now. If you can't see anything, please ensure that the ipyleaflet nbextension has been enabled.

## Setting the Area Of Interest (AOI)
Now that we have a location to centre the Map on, we need to determine the size of the map. This can be done using a range in latitude/longitude using the function below. This cell just defines how a function works, and does not perform any actual calculations. To do this we then call the function.

In [5]:
def expand_coords(centre,lon_expand = 1.0,lat_expand = 0.5):
    lat1 = centre[0]-lat_expand
    lat2 = centre[0]+lat_expand
    lon1 = centre[1]-lon_expand
    lon2 = centre[1]+lon_expand
    return [lon1,lat1,lon2,lat2,]
    

This next piece of code will then calculate the map size. If the map isn't the correct size for your liking, add into the function alternative settings for 'lon_expand' and 'lat_expand' from those set in the previous cell.

In [6]:
rec_coords = expand_coords(loc_coords)
# rec_coords = expand_coords(loc_coords,lat_expand=0.7) ## example adjusted rectangle size
local_rec = ee.Geometry.Rectangle(rec_coords) 

## Load Landsat data collections from GEE
Now we can see the the buffered AOI displayed on the Map. Next, let's load some Landsat images for the elected area. I've defined here a python function called `display_landsat_collection` to do so. It automatically loads both the [surface reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR) and [annual NDVI](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_ANNUAL_NDVI) image collections from GEE's data catalog and also calculates the annual means for each band. 

You can skip most of the details of what's inside the code cell, but only to look at the first (and last) line of code. In order to run such function, you will need to choose a year (any year since 1984) and an AOI. In the following example, I choose year 2019 and the Shenzhen buffer to demonstrate the use of the code.

In [1]:
landsat_vis_param = {
            'min': 0,
            'max': 3000,
            'bands': ['NIR', 'Red', 'Green']  # False Colour Composit bands to be visualised 
}
RGB_vis_param = {
            'min': 0,
            'max': 3000,
            'bands': [ 'Red', 'Green', 'Blue']  # False Colour Composit bands to be visualised 
}
ndvi_colorized_vis = {
            'min': 0.0,
            'max': 1.0,
            'palette': [
            'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
            '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
            '012E01', '011D01', '011301']
}

In [7]:
def load_landsat_collection(inmap,year, aoi, cloud_tolerance = 3.0, 
                            DISPLAY_ON_MAP = False, MEDIAN_ONLY = False):
    '''This function allows GEE to display a Landsat data collection 
    from any year between 1984 and present year
    that fall within the AOI and cloud tolerance, e.g. 3.0%.
    There are two optional flag:
    When DISPLAY_ON_MAP is TRUE, display this layer onto Map;
    When return_series = 'MEDIAN_ONLY', only median SR is loaded into landsat_ts, and
    Setting this option to MEDIAN_ONLY would be faster than loading other collections. 
    '''
    assert year >= 1984
    
    def renameBandsETM(image):
        if year >=2013: #LS8
            bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'] #, 'pixel_qa'
            new_bands = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'] #, 'pixel_qa'
        elif year <=1984:
            bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']
            new_bands = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
        return image.select(bands).rename(new_bands)
        
    if not(MEDIAN_ONLY):
        if year >= 2013:
            layer_name = 'LC08' # LS8: 2013-now        
        elif year == 2012: # # LS7: 1999- , however SLC error >= 1999:
            layer_name = 'LE07' 
        elif year >=1984:
            layer_name = 'LT05' # LS5: 1984-2012
       
        collection_name_sr = f"LANDSAT/{layer_name}/C01/T1_SR" 
        # You can also use the following line, if interested in incorperating ndvi
        collection_name_ndvi = f"LANDSAT/{layer_name}/C01/T1_ANNUAL_NDVI" 

        all_sr_image = ee.ImageCollection(collection_name_sr) \
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31') \
            .filter(ee.Filter.lt('CLOUD_COVER', cloud_tolerance))\
            .sort('system:time_start') \
            .select('B[1-7]') \
            .sort('CLOUD_COVER')

        all_sr_image = all_sr_image.map(renameBandsETM) # rename bands with 'renameBandsETM' function
        
        # reduce all_sr_image to annual average per pixel
        mean_image = all_sr_image.mean()
        mean_image = mean_image.clip(aoi).unmask()

        ndvi_image = ee.ImageCollection(collection_name_ndvi)\
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31')\
            .select('NDVI')\
            .median()
        ndvi_image = ndvi_image.clip(aoi).unmask()

        #mean_image.addBands(ndvi_image, 'NDVI')
    
    # This line loads all annual median surface ref
    landsat_ts = geemap.landsat_timeseries(roi=aoi, start_year=year, end_year=year, \
                                       start_date='01-01', end_date='12-31')

    median_image = landsat_ts.first().clip(aoi).unmask()
    
    if DISPLAY_ON_MAP == True:
        
        if not(MEDIAN_ONLY):
            inmap.addLayer(ndvi_image, ndvi_colorized_vis, 'NDVI '+str(year),  opacity=0.9)
            inmap.addLayer(mean_image, landsat_vis_param, "Mean Ref "+str(year))
        inmap.addLayer(median_image, landsat_vis_param, "Median Ref "+str(year))

    if MEDIAN_ONLY:
        return median_image
    else:
        return all_sr_image, mean_image, median_image, ndvi_image 

Now the `load_landsat_collection` function has been defined, and we will run/execute it by calling the function name with appropriate input parameters (or 'arguments). The output of such function will be returned to the variables on the LHS of the equal sign, i.e. all_image_2019, mean_2019, median_2019, and ndvi_2019 in this case.

In [8]:
Map = geemap.Map(center=loc_coords, zoom=10)

# All you need to modify here is the YEAR below:
all_image_2019, mean_2019, median_2019, ndvi_2019 = load_landsat_collection(Map,2019,\
                                        local_rec, cloud_tolerance = 2,\
                                        DISPLAY_ON_MAP = True)
Map

Map(center=[51.49163903397572, -0.08313179193795397], controls=(WidgetControl(options=['position', 'transparen…

You should examine the functions under the `toolbar` and `layer` buttons on the top-right corner of the Map, e.g. use the `inspector` and `plotting` tools to check the data values, or use `layers` control to switch on/off layers or to adjust opacity.

We can also check the metadata from the Landsat image collection we just loaded from the Cloud. Have a look of the output. Any useful information?

In [12]:
first_image = all_image_2019.first() 

props = geemap.image_props(first_image)
print( props.getInfo())

{'CLOUD_COVER': 0.01, 'CLOUD_COVER_LAND': 0.03, 'EARTH_SUN_DISTANCE': 0.989582, 'ESPA_VERSION': '2_23_0_1b', 'GEOMETRIC_RMSE_MODEL': 8.908, 'GEOMETRIC_RMSE_MODEL_X': 6.443, 'GEOMETRIC_RMSE_MODEL_Y': 6.152, 'IMAGE_DATE': '2019-02-24', 'IMAGE_QUALITY_OLI': 9, 'IMAGE_QUALITY_TIRS': 9, 'LANDSAT_ID': 'LC08_L1TP_201025_20190224_20190308_01_T1', 'LEVEL1_PRODUCTION_DATE': 1552088520000, 'NOMINAL_SCALE': 30, 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0', 'SATELLITE': 'LANDSAT_8', 'SENSING_TIME': '2019-02-24T10:52:30.0138839Z', 'SOLAR_AZIMUTH_ANGLE': 157.231583, 'SOLAR_ZENITH_ANGLE': 62.337173, 'SR_APP_VERSION': 'LaSRC_1.3.0', 'WRS_PATH': 201, 'WRS_ROW': 25, 'system:asset_size': '392.213325 MB', 'system:band_names': ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'], 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_201025_20190224', 'system:index': 'LC08_201025_20190224', 'system:time_end': '2019-02-24 10:52:30', 'system:time_start': '2019-02-24 10:52:30', 'system:version': 1563396284578517}


Next, examine the mean, median surface reflectance and/or NDVI layers we've visualized. Which one is better? How do these relate to the individual images?

One particular issue that can arrise is taking the 'mean' of individiual NDVI images. Instead we can take the 'NDVI' of mean images (or medians).


In addition to switching layers on and off, adjusting opacity, we can also use python code to some simple mathmatical operations, e.g. calculating the differences:

In [13]:
median_ndvi = median_2019.normalizedDifference(['NIR', 'Red'])
median_ndwi = median_2019.normalizedDifference(['Green','NIR'])

Map.addLayer(median_ndvi, ndvi_colorized_vis, 'NDVI from Median LS')
Map.addLayer(median_ndwi, ndvi_colorized_vis, 'NDWI from Median LS')
Map

Map(bottom=348954.0, center=[51.5072466571743, -0.028839111328125003], controls=(WidgetControl(options=['posit…

## Applying kernels to the data
Here we apply the spatial filtering techniques previously performed using ENVI (see [here](docs/SpatialFiltering.ipynb)) on the GEE images. The same method of convolving a kernel over an image is available, though we have to use the cloud based functional methods of Google Earth Engine. Some example kernels are shown below. What do these kernels do to the displayed data?

In [14]:
#### high and low pass 3x3 kernels as we used in ENVI. 
low_pass_K=ee.Kernel.fixed(3,3,[[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]],1,1,False)
high_pass_K=ee.Kernel.fixed(3,3,[[-1,-1,-1],[-1,8,-1],[-1,-1,-1]],1,1,False)
#### circle averaging kernel - in a functional form, but more flexible (try adjusting the radius)
low_pass_C=ee.Kernel.circle(radius = 5,units = 'pixels')

Map.addLayer(median_2019.convolve(high_pass_K),landsat_vis_param,'FCC high passed')
Map.addLayer(median_2019.convolve(low_pass_K),landsat_vis_param,'FCC low passed')
Map.addLayer(median_2019.convolve(low_pass_C),landsat_vis_param,'FCC low passed (circle)')

Map

Map(bottom=348987.0, center=[51.50019435946635, -0.0968170166015625], controls=(WidgetControl(options=['positi…

>## Advanced Challenge
>Can you copy the cell above, and adjust the code to add a high passed version of a visual coloured median image? You'll need the `RGB_vis_param` display specifications, and the `median_2019` image.
>
> Can you create a low passed version of the NDVI image?

## Mapping regions with high Vegetation or Water Index
When we used ENVI we used ranges of pixel values to select certain areas. This can be done with GEE too, although it is a littel more tricky. The code below will create maps that split the pixels depending upon:
- NDVI > 0.5
- NDWI > 0.0

And then by combining the opposite of the previous two maps
- NDVI < 0.5 AND NDWI < 0.0

In [17]:
### NDVI > 0.5
ndvi_gt = median_ndvi.gt(median_ndvi.constant(0.5))
Map.addLayer(ndvi_gt,{},'NDVI > 0.5')

### NDWI > 0.0
ndwi_gt=median_ndwi.gt(median_ndwi.constant(0.0))
Map.addLayer(ndwi_gt,{},'NDWI > 0.0')

### Opposite of BOTH
both_lt=ndwi_gt.Not().And(ndvi_gt.Not())
Map.addLayer(both_gt,{},'NDVI < 0.5, AND NDWI < 0.0')
Map

Map(bottom=349057.0, center=[51.485231326900056, -0.14453887939453128], controls=(WidgetControl(options=['posi…

## Majority Analysis
For a highly variable finely detailed classification, such as we performed above, the resulting maps can appear noisey, and contain data that is not useful for large scale mean metrics. We can remove this detail by applying a majority kernel. The code below will first apply this to the NDVI > 0.5 map. 
>Can you adjust the code (add and remove `#` characters) to apply the different kernels?
>
>Can you apply the majority kernel to the BOTH map created previously?

Compare the maps using the layer selection tool, how do they compare at different zoom levels?

In [18]:
majority_ndvi = ndvi_gt.reduceNeighborhood(
                reducer=ee.Reducer.mode(), 
                kernel=ee.Kernel.fixed(3,3,[[1]*3]*3,1,1,False) #3x3
#                 kernel=ee.Kernel.fixed(9,9,[[1]*9]*9,5,5,False) #9x9
#                 kernel=ee.Kernel.circle(4)
                )
Map.addLayer(majority_ndvi,{},'NDVI > 0.5, 3x3 majority')
Map

Map(bottom=698140.0, center=[51.45304417416522, -0.08394241333007814], controls=(WidgetControl(options=['posit…